# Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split

# Read Data

In [2]:
iris_data = pd.read_csv(os.getcwd()+"/Data/Iris.csv")
iris_data=iris_data.iloc[np.random.permutation(len(iris_data))].reset_index(drop=True)
# iris_data = iris_data_.head(100).reset_index(drop=True)
# iris_data_test = iris_data_.tail(50).reset_index(drop=True)

In [3]:
def normalization(iris_data):
    species_dic = {'Iris-setosa':[1,0,0], 'Iris-versicolor':[0,1,0], 'Iris-virginica':[0,0,1]}
    iris_data['Species_'] = iris_data['Species'].map(lambda x: species_dic[x])
    #Normalization
    for col in ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']:
        iris_data[col] = (iris_data[col] - iris_data[col].min())/ (iris_data[col].max() - iris_data[col].min())
    return iris_data

In [4]:
iris_data = normalization(iris_data)
# iris_data_test = normalization(iris_data_test)

# Three Layer Neural Network

In [5]:
Weights_layer1 = np.random.randn(4, 8)
Weights_layer2 = np.random.randn(8, 5)
Weights_layer3 = np.random.randn(5, 3)


Bias_layer1 = np.zeros((1, 8))
Bias_layer2 = np.zeros((1, 5))
Bias_layer3 = np.zeros((1, 3))

In [6]:
Weights_layer3

array([[ 1.85411936,  1.28853836,  2.61343797],
       [-0.75745208,  0.51848065, -0.49031599],
       [-0.73228694,  0.05253693,  0.79872181],
       [ 0.09544274, -0.03481573, -0.14658841],
       [ 1.20442419,  0.80603329, -0.84877513]])

In [7]:
Bias_layer3

array([[0., 0., 0.]])

In [8]:
def activation_function(x):
    return 1/(1+np.exp(-x))
#     return np.maximum(0, x)

In [9]:
def activation_function_der(x):
    return x*(1-x)
#     return np.where(x > 0, 1, 0)

In [10]:
def feed_farword(X):
    output_layer0 = X
    output_layer1 = activation_function(np.matmul(X, Weights_layer1) + Bias_layer1)  
    output_layer2 = activation_function(np.matmul(output_layer1, Weights_layer2) + Bias_layer2) # (1,8) * (8,5) + (1,5)
    output_layer3 = activation_function(np.matmul(output_layer2, Weights_layer3) + Bias_layer3) # (1,5) * (5,3) + (1,3)
    return (output_layer0, output_layer1, output_layer2, output_layer3)

In [11]:
def back_propagation(Y, output_layer0, output_layer1, output_layer2, output_layer3):
    global Weights_layer1, Weights_layer2, Weights_layer3, Bias_layer1, Bias_layer2, Bias_layer3
    error = 0.33*sum(sum((Y - output_layer3)**2))
    
    
    gradient_layer3 = np.multiply(-1*(Y - output_layer3), activation_function_der(output_layer3)) #(1,3)
    Weights_layer3 = Weights_layer3 - learning_rate * np.matmul(gradient_layer3.T, output_layer2).T
    Bias_layer3 = Bias_layer3 - learning_rate * gradient_layer3
    
    
    gradient_layer2 = np.multiply(np.matmul(gradient_layer3, Weights_layer3.T), activation_function_der(output_layer2))
    Weights_layer2 = Weights_layer2 - learning_rate * np.matmul(gradient_layer2.T, output_layer1).T
    Bias_layer2 = Bias_layer2 - learning_rate * gradient_layer2
    
    
    gradient_layer1 = np.multiply(np.matmul(gradient_layer2, Weights_layer2.T), activation_function_der(output_layer1))
    Weights_layer1 = Weights_layer1 - learning_rate * np.matmul(gradient_layer1.T, output_layer0).T
    Bias_layer1 = Bias_layer1 - learning_rate * gradient_layer1
    
    
    return error

In [12]:
def get_accuracy(iris_data, output_layer3):
    Predicted_Species = []
    species_dic = {'Iris-setosa':[1,0,0], 'Iris-versicolor':[0,1,0], 'Iris-virginica':[0,0,1]}
    for i in range(0, output_layer3.shape[0]):
        Predicted_Species.append(list(species_dic.keys())[np.argmax(output_layer3[i])])
    iris_data['Predicted_Species'] = Predicted_Species
    return (iris_data[iris_data['Species']==iris_data['Predicted_Species']].shape[0]/iris_data.shape[0])

In [13]:
X = iris_data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
Y = iris_data[['Species_']]

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)


X=np.array(X) #(150, 4)
Y = Y.values
Y = [i[0] for i in Y]
Y = np.array(Y) # (150,3)

In [14]:
errors = []
epoch = 5000
learning_rate = 0.01
for i in range(0, epoch):
    for j in range(0, X.shape[0]):
        output_layer0, output_layer1, output_layer2, output_layer3 = feed_farword(X[j,].reshape((1, 4)))
        error = back_propagation(Y[j,], output_layer0, output_layer1, output_layer2, output_layer3)
    if i%1000==0:
        output_layer0, output_layer1, output_layer2, output_layer3 = feed_farword(X)
        print(error, get_accuracy(iris_data, output_layer3))

0.2806092083688256 0.3333333333333333
0.004710438003499211 0.98
0.0013321030483074878 0.9866666666666667
0.0007801186199264969 0.9866666666666667
0.0005388898296986991 0.9866666666666667


In [15]:
output_layer0, output_layer1, output_layer2, output_layer3 = feed_farword(X)

In [16]:
Predicted_Species = []
prop = []
species_dic = {'Iris-setosa':[1,0,0], 'Iris-versicolor':[0,1,0], 'Iris-virginica':[0,0,1]}
for i in range(0, output_layer3.shape[0]):
    Predicted_Species.append(list(species_dic.keys())[np.argmax(output_layer3[i])])
    prop.append(output_layer3[i].max())
iris_data['Predicted_Species'] = Predicted_Species
iris_data['Probability'] = prop


In [17]:
iris_data.to_excel(os.getcwd()+"/Data/Iris_ANN_output.xlsx")